In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [7]:
response = requests.get("https://www.alfabeta.net/precio/abarax.html") #, headers={"Accept-Language":"en-US"}
soup = BeautifulSoup(response.content, "html.parser")

# Como obtener droga laboratorio y tipo

In [ ]:
for movie in soup.find_all("span", class_="defecto"):
    print(movie)

In [9]:
response = requests.get("https://www.alfabeta.net/precio/actron.html") #, headers={"Accept-Language":"en-US"}
soup = BeautifulSoup(response.content, "html.parser")

In [10]:
for movie in soup.find_all("span", class_="defecto"):
    print(movie)

<span class="defecto">Bayer Consumer</span>
<span class="defecto" style="word-break:break-all;"><i>ibuprofeno</i></span>
<span class="defecto"><i>Analgésico Antiinflam.</i></span>


In [19]:
lab = soup.find_all("span", class_="defecto")[0].string
drug = soup.find_all("span", class_="defecto")[1].string
d_type = soup.find_all("span", class_="defecto")[2].string

# Como obtener el resto de los datos

In [39]:
description = soup.find_all("table", class_="presentacion")[0].find("td", class_="tddesc")
description.get_text()

'caps.gelat.blanda x 10 (VL)'

In [40]:
price = soup.find_all("table", class_="presentacion")[0].find("td", class_="tdprecio")
float(price.get_text().replace("$",""))

256.0

In [ ]:
#tddesc #tdprecio
a_row = pd. Series([1, 2])
df = pd. DataFrame([[3, 4], [5, 6]])
row_df = pd. DataFrame([a_row])
df = pd. concat([row_df, df], ignore_index=True)
print(df)

In [51]:
df = []
for dformat in soup.find_all("table", class_="presentacion"):
    price = float(dformat.find("td", class_="tdprecio").get_text().replace("$","")) 
    description = dformat.find("td", class_="tddesc")
    df.append([lab, d_type, drug, description, price])
    
df = pd.DataFrame(df)

In [52]:
df

,0,1,2,3,4
0,Bayer Consumer,Analgésico Antiinflam.,ibuprofeno,"[caps.gelat.blanda x 10, [ (VL)]]",256.0
1,Bayer Consumer,Analgésico Antiinflam.,ibuprofeno,"[caps.gelat.blanda x 20, [ (VL)]]",489.0


In [6]:
def get_brand_df(brand):
    proxy = proxylist[random.randint(0, len(proxylist))]
    response = requests.get(f'https://www.alfabeta.net/precio/{brand}.html',proxies={'http': proxy, 'https': proxy}) #, headers={"Accept-Language":"en-US"}
    soup = BeautifulSoup(response.content, "html.parser")
    df = []
    
    if len(soup.find_all("span", class_="defecto")) == 3:
        lab = soup.find_all("span", class_="defecto")[0].string
        drug = soup.find_all("span", class_="defecto")[1].string
        d_type = soup.find_all("span", class_="defecto")[2].string
        for dformat in soup.find_all("table", class_="presentacion"):
            price = float(dformat.find("td", class_="tdprecio").get_text().replace("$","")) 
            description = dformat.find("td", class_="tddesc").get_text()
            df.append([lab, d_type, drug, description, price])

    return pd.DataFrame(df)

In [7]:
get_brand_df('actron')

ProxyError: HTTPSConnectionPool(host='www.alfabeta.net', port=443): Max retries exceeded with url: /precio/actron.html (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(54, 'Connection reset by peer')))

In [8]:
get_brand_df('dolten')

ProxyError: HTTPSConnectionPool(host='www.alfabeta.net', port=443): Max retries exceeded with url: /precio/dolten.html (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x16dd6f070>: Failed to establish a new connection: [Errno 60] Operation timed out')))

In [5]:
import requests
from bs4 import BeautifulSoup
import random
import concurrent.futures 

In [ ]:
#get the list of free proxies
def getProxies():
    r = requests.get('https://free-proxy-list.net/')
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find('tbody')
    proxies = []
    for row in table:
        if row.find_all('td')[4].text =='elite proxy':
            proxy = ':'.join([row.find_all('td')[0].text, row.find_all('td')[1].text])
            proxies.append(proxy)
        else:
            pass
    return proxies

def extract(proxy):
    #this was for when we took a list into the function, without conc futures.
    #proxy = random.choice(proxylist)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0'}
    try:
        #change the url to https://httpbin.org/ip that doesnt block anything
        r = requests.get('https://httpbin.org/ip', headers=headers, proxies={'http' : proxy,'https': proxy}, timeout=1)
        print(r.json(), r.status_code)
    except requests.ConnectionError as err:
        print(repr(err))
    return proxy

proxylist = getProxies()
#print(len(proxylist))

#check them all with futures super quick
with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(extract, proxylist)

In [11]:
get_brand_df('nasonex')

,0,1,2,3,4
0,Organon Arg.,Antialérgico Antiinflamatorio,mometasona furoato,Nas.spray acuoso x140ds.,4526.15


# Como obtener la lista de brands

In [40]:
brands_df = pd.read_csv('brands.csv')

In [41]:
brands_df

,brands
0,actron
1,tafirol
2,abarax


In [43]:
brands = brands_df['brands'].to_numpy()

In [45]:
!ls

brands.csv  elea.html  pfizer.html  Untitled.ipynb


# Sacar las marcas de las paginas de los laboratorios

In [46]:
pfizer_page = open('pfizer.html')
pfizer_soup = BeautifulSoup(pfizer_page.read())

In [92]:
list_of_labs = ['bago', 'elea', 'pfizer']

In [83]:
list_of_labs = ['elea']

In [94]:
def get_brands(list_of_labs):
    brands = []
    for lab in list_of_labs:
        page = open(f'{lab}.html')
        soup = BeautifulSoup(page.read())
        for brand in pfizer_soup.find_all("a", class_="rprod"):
            brands.append(brand.string.split(" ")[0].lower())
        
    return brands

In [96]:
list_of_brands = list(set(get_brands(list_of_labs)))

In [97]:
list_of_brands

['azulfidine',
 'celebrex',
 'vibramicina',
 'isordil',
 'trapax',
 'depo-provera',
 'ixifi',
 'xalatan',
 'zoloft',
 'nordette',
 'terra',
 'talzenna',
 'synrelin',
 'fasigyn',
 'arteriopril',
 'xanax',
 'dolten',
 'tigify',
 'vyndaqel',
 'pergamid',
 'ibrance',
 'lipitor',
 'aldactone',
 'respimex',
 'v',
 'ibumax',
 'benefix',
 'rosuvastatina',
 'ibupiretas',
 'genotropin',
 'viagra',
 'abrilada',
 'champix',
 'dalacin',
 'lopid',
 'detrusitol',
 'rapamune',
 'losartan',
 'nimenrix',
 'pristiq',
 'diabinese',
 'prolisina',
 'dostinex',
 'eliquis',
 'trazimera',
 'sutent',
 'zyvox',
 'farlutale',
 'micronema',
 'unasyna',
 'inlyta',
 'artane',
 'somavert',
 'trosyd',
 'vyndamax',
 'xyntha',
 'vizimpro',
 'ronfase',
 'lyrica',
 'zavedos',
 'cardura',
 'zavicefta',
 'tramadolten',
 'aromasin',
 'prevenar',
 'solu',
 'farmorubicin',
 'merozen',
 'ruxience',
 'zeldox',
 'cabaser',
 'enbrel',
 'premarin',
 'minodiab',
 'bosulif',
 'mucoprednibron',
 'besponsa',
 'zinforo',
 'ibupirac',
 '

In [86]:
def get_df(list_of_brands):
    df = pd.DataFrame()
    for brand in list_of_brands:
        brand_df = get_brand_df(brand)
        pd.concat([df, brand_df])
    return df

In [87]:
df = get_df(list_of_brands)

""
